# RAG com LCEL

Agora vamos para exemplos mais avançados de chains e como podemos reproduzi-las utilizando LCEL.

Vamos começar mostrando como fazer o processo de RAG. Para isso, o processo de criação da vectorstore é exatamente igual ao que vimos no curso de Aplicações de IA com LangChain:

In [3]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [4]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
paths = [
    "assets/Explorando o Universo das IAs com Hugging Face.pdf",
    ]

pages = []

for path in paths:
    loader = PyPDFLoader(path)
    pages.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(pages)

c:\Users\annaf\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [6]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [7]:
from langchain_community.vectorstores.faiss import FAISS

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embeddings_model
)
retriever = vectorstore.as_retriever(search_type='mmr')

Agora na criação da chain é que temos uma mudança. Podemos definir nossa chain manualmente da seguinte forma:

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI()

template_str = '''Responda as seguintes perguntas do usuário utilizando apenas o contexto fornecido.

Contexto: {contexto}

Pergunta: {pergunta}'''

template = ChatPromptTemplate.from_template(template_str)
output_parser = StrOutputParser()

In [9]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup_and_retrievel = RunnableParallel(
    {'pergunta': RunnablePassthrough(),
     'contexto': retriever}
)

chain = setup_and_retrievel | template | model | output_parser

In [10]:
chain.invoke('O que é o Hugging Face?')

'O Hugging Face é uma empresa que iniciou em 2017 na França, com o desenvolvimento de Chatbots.'

## Entendendo o RunnableParallel e o RunnablePassthrough

In [11]:
{'pergunta': RunnablePassthrough().invoke('O que é o Hugging Face?')}

{'pergunta': 'O que é o Hugging Face?'}

In [12]:
retriever.invoke('O que é o Hugging Face?')

[Document(page_content='Explorando o Universo das IAs com Hugging Face\n01. O que é Hugging Face?\nBem-vindos ao curso de Hugging Face da Asimov Academy!\nNeste curso, vamos explorar as principais utilidades da plataforma de IA Hugging Face . Aprenderemos\ncomo utilizar a plataforma ao máximo, e como incorporar as bibliotecas de Python do Hugging Face\naos nossos scripts de Python.\nAfinal, o que é Hugging Face?\nA Hugging Face é uma empresa que iniciou em 2017 na França, com o desenvolvimento de Chatbots.', metadata={'source': 'assets/Explorando o Universo das IAs com Hugging Face.pdf', 'page': 5}),
 Document(page_content='Face.\nÉ claro que o Hugging Face não acaba aqui. Não tivemos tempo de explorar boa parte das tarefas que\nexistem no Hugging Face. Dito isso, agora sabemos como fazer este processo: conseguimos procurar por\numa tarefa nova, encontrar o código necessário para baixá-lo (seja pela biblioteca transformers\nou alguma outra), avaliar se conseguiremos rodar o modelo loca

In [13]:
RunnableParallel(
    {'pergunta': RunnablePassthrough(),
     'contexto': retriever}
).invoke('O que é o Hugging Face?')

{'pergunta': 'O que é o Hugging Face?',
 'contexto': [Document(page_content='Explorando o Universo das IAs com Hugging Face\n01. O que é Hugging Face?\nBem-vindos ao curso de Hugging Face da Asimov Academy!\nNeste curso, vamos explorar as principais utilidades da plataforma de IA Hugging Face . Aprenderemos\ncomo utilizar a plataforma ao máximo, e como incorporar as bibliotecas de Python do Hugging Face\naos nossos scripts de Python.\nAfinal, o que é Hugging Face?\nA Hugging Face é uma empresa que iniciou em 2017 na França, com o desenvolvimento de Chatbots.', metadata={'source': 'assets/Explorando o Universo das IAs com Hugging Face.pdf', 'page': 5}),
  Document(page_content='Face.\nÉ claro que o Hugging Face não acaba aqui. Não tivemos tempo de explorar boa parte das tarefas que\nexistem no Hugging Face. Dito isso, agora sabemos como fazer este processo: conseguimos procurar por\numa tarefa nova, encontrar o código necessário para baixá-lo (seja pela biblioteca transformers\nou algum

No final o que estamos fazendo ao utilizarmos o RunnableParallel é criar uma estrutura de dicionários cujo os valores são gerados em paralelo!

## Um alternativa não paralelizável

Da seguinte forma, também obeteríamos o mesmo resultado, mas de forma não paralelizada, o que pode gerar um atraso no processamento da nossa chain.

In [14]:
setup_dict = {'pergunta': RunnablePassthrough(), 'contexto': retriever}
chain = setup_dict | template | model | output_parser
chain.invoke('O que é o Hugging Face?')

'O Hugging Face é uma empresa que iniciou em 2017 na França, com o desenvolvimento de Chatbots.'

# Fallbacks

Ao trabalhar com modelos de linguagem, você pode frequentemente encontrar problemas nas APIs subjacentes, seja por limitação de taxa ou tempo de inatividade. Portanto, à medida que você move suas aplicações LLM para produção, torna-se cada vez mais importante proteger-se contra esses problemas. É por isso que introduzimos o conceito de **fallbacks**, ou alternativas em português.

Uma alternativa é um plano substituto que pode ser usado em uma emergência.

Criticamente, as alternativas podem ser aplicadas não apenas no nível do LLM, mas em todo o nível executável. Isso é importante porque, muitas vezes, modelos diferentes exigem prompts diferentes. Então, se sua chamada para a OpenAI falhar, você não quer simplesmente enviar o mesmo prompt para a Anthropic - você provavelmente vai querer usar um template de prompt diferente e enviar uma versão diferente lá.

## Fallback para entradas grandes

Quando construímos aplicações, precisamos sempre atentar às questões economicas que envolvem colocar um modelo em produção. Muitas vezes será necessário otimizar nossos custos, evitando utilizar modelos maiores (e consequentemente mais caros) para problemas simples. Com fallback, temos a alternativa de sempre tentar processar a entrada do usuário com um modelo menor e, caso tivermos um erro no processamento, tentamos com um modelo mais oneroso. Mostramos aqui um exemplo onde utilizamos um modelo mais simples com context window menor que, no caso de uma entrada maior, é suubstituido por um modelo mais complexo.

In [16]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(model='gpt-3.5-turbo-instruct')
prompt = PromptTemplate.from_template('Resuma o seguinte texto: {texto}')


chain_pequena = prompt | llm
chain_pequena.invoke({'texto': 'Oi, eu sou a Anna'})


'\n\nAnna se apresenta.'

In [17]:
chain_pequena.invoke({'texto': 'UM TEXTO REPETIDO' * 1000})

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 6263 tokens (6007 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [18]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model='gpt-3.5-turbo-0125')
prompt = ChatPromptTemplate.from_template('Resuma o seguinte texto: {texto}')
chain_grande = prompt | model | StrOutputParser()
chain_grande.invoke({'texto': 'UM TEXTO REPETIDO' * 1000})

'O texto é repetido várias vezes.'

In [19]:
chain_fallback = chain_pequena.with_fallbacks([chain_grande])
chain_fallback.invoke({'texto': 'UM TEXTO REPETIDO' * 1000})

'O texto é repetido várias vezes.'

In [20]:
chain_fallback.invoke({'texto': 'UM TEXTO REPETIDO'})

'\n\nO texto em questão é um repetido.'

## Fallbacks para formatações

Da mesma forma ocorre na formatação de informação, aplicação muito corrente para modelos de linguagem. Em geral, podemos utilizar modelos simples para essas funções mas, seguidamente, pelas suas limitações eles acabam errando. Nestes casos, podemos utilizar fallbacks para gerar uma formatação com modelos mais complexos quando houver erros.

In [21]:
from langchain_openai import ChatOpenAI, OpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import DatetimeOutputParser

prompt = PromptTemplate.from_template(
    "Qual foi a data e hora do evento {evento} (no formato %Y-%m-%dT%H:%M:%S.%fZ - retorne apenas o valor solicitado)"
)

In [22]:
chain_instruct = prompt | OpenAI(model='gpt-3.5-turbo-instruct') | DatetimeOutputParser()
chain_instruct.invoke({'evento': 'A final da compa do mundo de 2002'})

datetime.datetime(2002, 6, 30, 0, 0)

In [23]:
chain_turbo = prompt | ChatOpenAI(model='gpt-3.5-turbo') | DatetimeOutputParser()
chain_turbo.invoke({'evento': 'A final da compa do mundo de 2002'})

datetime.datetime(2002, 6, 30, 19, 30)

In [24]:
chain_fallback = chain_instruct.with_fallbacks([chain_turbo])
chain_fallback.invoke({'evento': 'A final da compa do mundo de 2002'})

datetime.datetime(2002, 6, 30, 0, 0)